In [1]:
!export HF_HOME=/shared/data3/pk36/.cache

In [31]:
beta = 2
gamma = 3
breadth_weight = 0.5
pos_rank = 1
breadth = 0.7 # mean
depth = 0.8 # max

(beta * pos_rank)/(gamma * (breadth_weight*breadth + (1-breadth_weight)*depth))

0.8888888888888888

In [2]:
%load_ext autoreload
%autoreload 2
from PyPDF2 import PdfReader
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"
os.environ['HF_HOME'] = '/shared/data3/pk36/.cache'
import shutil
import subprocess
import re
from utils import *

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
class Args:
    def __init__(self):
        self.dataset = "vaccine"
        self.topic = "Which COVID-19 vaccine is better, Pfizer or Moderna?"
        self.override = False
        self.iters = 4
        self.split = False
        self.model = "biobert"
        self.override = True
        self.dim = 768
        self.length = 512

args = Args()

**Identify Aspects & Key Topics**

In [4]:
aspects, keywords = aspect_candidate_gen(args.topic, k=5)

/home/pk36/inverse_knowledge_search/inverse/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pk36/inverse_knowledge_search/inverse/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Here are the 5 aspects and their corresponding keywords for the topic "Which COVID-19 vaccine is better, Pfizer or Moderna?":

---
aspect_0: Efficacy
aspect_0_keywords: efficacy rate, vaccine effectiveness, protection rate, immune response, clinical trials, vaccine potency, disease prevention, infection rate, transmission reduction, public health
aspect_1: Safety
aspect_1_keywords: adverse reactions, side effects, allergic reactions, anaphylaxis, serious adverse events, vaccine safety, immunogenicity, tolerability, reactogenicity, pharmacovigilance
aspect_2: Cost
aspect_2_keywords: cost-effectiveness, pricing strategy, out-of-pocket cost, insurance coverage, healthcare system, economic burden, budget impact, affordability, value-based pricing, reimbursement
aspect_3: Convenience
aspect_0_keywords: administration route, dosing schedule, storage requirements, transportation, handling, distribution, logistics, vaccination site, patient compliance, user experience
aspect_4: Acceptance
aspe

**Utilize these phrases to augment the "expert" phrases for AutoPhrase before we pre-process it**

In [6]:
with open("preprocessing/AutoPhrase/data/EN/wiki_quality_orig.txt", "r") as f:
    all_phrases = [w.strip() for w in f.readlines()]

    for a in aspects:
        all_phrases.append(a.replace("_", " "))

    all_phrases.extend([w.replace("_", " ") for a in keywords for w in a])

In [7]:
with open("preprocessing/AutoPhrase/data/EN/wiki_quality.txt", "w") as f:
    for w_id, w in enumerate(all_phrases):
        if w_id == (len(all_phrases) - 1):
            f.write(f"{w}")
        else:
            f.write(f"{w}\n")

In [ ]:
# printable = set(string.printable)
# with open('datasets/qa_papers.jsonl', 'r', encoding='utf-8', errors='ignore') as json_file:
#     json_list = list(json_file)

#     results = []
#     for json_str in json_list:
#         results.append(json.loads(json_str))


# with open("datasets/qa_papers.txt", "w", encoding='utf-8', errors='ignore') as f:
#     for result in tqdm(results):
#         result['Content'] = ''.join(filter(lambda x: x in printable, result['Content'])).replace('\r', '')
#         f.write(f"Title: {result['Title']}; Abstract: {result['Abstract']}; Paper: {result['Content'].split(' References ')[0]}")
#         if result != results[-1]:
#             f.write("\n")

**Read in Dataset & Pre-process**

In [8]:
if args.override or (not os.path.exists(f"datasets/{args.dataset}/phrase_{args.dataset}.txt")):
    if not os.path.exists(f"datasets/{args.dataset}/{args.dataset}_text.txt"):
        pdf_files = []
        for file in os.listdir(f"datasets/{args.dataset}"):
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(f"datasets/{args.dataset}", file))
        
        with open(f"datasets/{args.dataset}/{args.dataset}_text.txt", "w") as f:
            for file in pdf_files:
                reader = PdfReader(file)
                paper = []
                for page in reader.pages:
                    page_text = page.extract_text()
                    page_text = re.sub(fr"\s+", " ", page_text)
                    paper.append(page_text)
                f.write(f"{' '.join(paper)}\n")

        shutil.copyfile(f"datasets/{args.dataset}/{args.dataset}_text.txt", f"datasets/{args.dataset}/text.txt")

    # pre-process
    os.chdir("./preprocessing")
    subprocess.check_call(['./auto_phrase.sh', args.dataset])
    os.chdir("../")
else:
    print("already pre-processed!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


===Corpus Pre-processing===


100%|██████████| 50/50 [00:02<00:00, 21.46it/s]


===Compilation===
===Tokenization===



real	0m1.719s
user	0m14.568s
sys	0m0.575s


...

Detected Language: EN
===Part-Of-Speech Tagging===
Current step: Merging...s...
===Phrasal Segmentation===


=== Current Settings ===
Segmentation Model Path = models/NEW/segmentation.model
After the phrasal segmentation, only following phrases will be highlighted with <phrase> and </phrase>
	Q(multi-word phrases) >= 0.700000
	Q(single-word phrases) >= 1.000000
POS guided model loaded.
# of loaded patterns = 5210
# of loaded truth patterns = 48646
POS transition matrix loaded
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't have enough POS tags
POS file doesn't hav

===Generating Output===
===Segmented Corpus Post-processing===


50it [00:00, 796.14it/s]


Phrase segmented corpus written to ../datasets/vaccine/phrase_vaccine.txt


**Seed-guided topic discovery based on the aspects**

In [18]:
# format the input keywords file for seetopic -> get phrases -> filter using LLM
dir_name = args.dataset.replace(" ", "_")

if not os.path.exists(f"SeeTopic/{dir_name}"):
    os.makedirs(f"SeeTopic/{dir_name}")

if args.split:
    chunk_len = 100 # words
    window = 10 # words
    with open(f"datasets/{args.dataset}/phrase_{args.dataset}.txt", "r") as fin, open(f"SeeTopic/{dir_name}/{dir_name}.txt", "w") as fout:
        lines = [l.strip() for l in fin]
        for line in lines:
            all_words = line.split(" ")
            for chunk in range(0, len(all_words), chunk_len):
                if chunk == 0:
                    fout.write(f"{' '.join(all_words[chunk:chunk+chunk_len])}\n")
                else:
                    fout.write(f"{' '.join(all_words[chunk-window:chunk+chunk_len])}\n")

else:
    shutil.copyfile(f"datasets/{args.dataset}/phrase_{args.dataset}.txt", f"SeeTopic/{dir_name}/{dir_name}.txt")

## aspects and seed terms
with open(f"SeeTopic/{dir_name}/keywords_0.txt", "w") as f:
    for idx, c in enumerate(zip(aspects, keywords)):
        str_c = ",".join(c[1])
        f.write(f"{idx}:{c[0]},{str_c}\n")

In [19]:
with open(os.path.join(f"SeeTopic/{args.dataset}", f"{dir_name}.txt")) as fin:
    lines = [l.strip() for l in fin]

In [20]:
# os.chdir("../")
os.chdir("./SeeTopic")
subprocess.check_call(['./seetopic.sh', args.dataset, str(args.iters), args.model])
os.chdir("../")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


===Get PLM Embeddings===
####### CONSTRUCTING AND TOKENIZING VOCAB #######
####### COMPUTING STATIC EMBEDDINGS #######


100%|██████████| 10407/10407 [01:18<00:00, 133.22it/s]


===Iter 0: PLM Module===
===Iter 1: PLM Module===
===Iter 1: Local Module===
make: 'cate' is up to date.
Starting training using file ../vaccine/vaccine.txt
Reading topics from file vaccine_1/keywords.txt
Vocab size: 10371
Words in train file: 394050
Read 5 topics
clinical_trials	eradicate	vaccine_induced	
adverse_reactions	side_effects	allergic_reactions	
costco	cost_effective	insured	
storage_requirements	transportation	handling	
trust	social_media	attitude	
Pre-training for 2 epochs, in total 2 + 10 = 12 epochs
Alpha: 0.000285  Progress: 99.10%  Words/thread/sec: 14.09k  Topic mining results written to file vaccine_1/res_cate.txt
===Iter 1: Ensemble===
===Iter 2: PLM Module===
===Iter 2: Local Module===
make: 'cate' is up to date.
Starting training using file ../vaccine/vaccine.txt
Reading topics from file vaccine_2/keywords.txt
Vocab size: 10371
Words in train file: 394050
Read 5 topics
clinical_trials	eradicate	vaccine_induced	cell_mediated	infectivity	neutralize	
adverse_reaction